## Ajout de caracteristiques :
Nous allons ajouter certaines caracteristiques qui semblent coherentes avec le probleme mais aussi qui peuvent aider a predire le resultat d'un match.

### Forme recente

In [59]:
import pandas as pd

In [60]:
df = pd.read_csv("/content/premier_league_nettoye.csv")
df.head()

,date_match,equipe_dom,equipe_ext,buts_dom,buts_ext,resultat,tirs_dom,tirs_ext,tirs_cadres_dom,tirs_cadres_ext,jaunes_dom,jaunes_ext,rouges_dom,rouges_ext,cible_resultat,season
0,10/08/2018,Man United,Leicester,2,1,H,8,13,6,4,2,1,0,0,2,2018-2019
1,11/08/2018,Bournemouth,Cardiff,2,0,H,12,10,4,1,1,1,0,0,2,2018-2019
2,11/08/2018,Fulham,Crystal Palace,0,2,A,15,10,6,9,1,2,0,0,0,2018-2019
3,11/08/2018,Huddersfield,Chelsea,0,3,A,6,13,1,4,2,1,0,0,0,2018-2019
4,11/08/2018,Newcastle,Tottenham,1,2,A,15,15,2,5,2,2,0,0,0,2018-2019


Maintenant on va s'occuper de gerer la date

In [61]:
df["date_match"] = pd.to_datetime(df["date_match"], dayfirst=True, errors='coerce')


In [62]:
df = df.sort_values("date_match").reset_index(drop=True)


In [63]:
df[["date_match", "equipe_dom", "buts_dom"]].head(10)
df[["date_match", "equipe_dom", "buts_dom"]].tail(10)


,date_match,equipe_dom,buts_dom
2270,2024-05-19,Liverpool,2
2271,2024-05-19,Crystal Palace,5
2272,2024-05-19,Luton,2
2273,2024-05-19,Chelsea,2
2274,2024-05-19,Man City,3
2275,2024-05-19,Brighton,0
2276,2024-05-19,Brentford,2
2277,2024-05-19,Arsenal,2
2278,2024-05-19,Burnley,1
2279,2024-05-19,Sheffield United,0


On ajoute un systeme de points par match

In [64]:
def calcul_points(row):
    if row["buts_dom"] > row["buts_ext"]:
        return 3
    elif row["buts_dom"] < row["buts_ext"]:
        return 0
    else:
        return 1

df["points_dom"] = df.apply(calcul_points, axis=1)
df["points_ext"] = 3 - df["points_dom"]


On transforme notre dataset pour que chaque ligne corresponde a une equipe

In [65]:
home_df = df[[
    "date_match", "equipe_dom",
    "buts_dom", "buts_ext",
    "points_dom",
    "tirs_dom", "tirs_cadres_dom"
]].copy()

home_df.columns = [
    "date_match", "equipe",
    "buts_pour", "buts_contre",
    "points",
    "tirs", "tirs_cadres"
]

away_df = df[[
    "date_match", "equipe_ext",
    "buts_ext", "buts_dom",
    "points_ext",
    "tirs_ext", "tirs_cadres_ext"
]].copy()

away_df.columns = home_df.columns

team_df = pd.concat([home_df, away_df])


Pour ce qui est de la forme nous allons prendre en compte les 5 derniers matchs

In [66]:
def rolling_features(df_team, col_list, window=5):
    rolling_df = pd.DataFrame()

    for team in df_team["equipe"].unique():
        team_matches = df_team[df_team["equipe"] == team].sort_values("date_match").copy()
        for col in col_list:
            team_matches[f"{col}_moy_{window}"] = team_matches[col].rolling(window, min_periods=1).mean().shift(1)
        rolling_df = pd.concat([rolling_df, team_matches])

    rolling_df = rolling_df.sort_values(["date_match", "equipe"]).reset_index(drop=True)
    return rolling_df


In [67]:
rolling_cols = ["buts_pour", "buts_contre", "points", "tirs"]
team_df = rolling_features(team_df, rolling_cols, window=5)


In [68]:
team_df.index.is_unique
team_df.head(10)
team_df.tail(10)


,date_match,equipe,buts_pour,buts_contre,points,tirs,tirs_cadres,buts_pour_moy_5,buts_contre_moy_5,points_moy_5,tirs_moy_5
4550,2024-05-19,Liverpool,2,0,3,36,14,2.4,2.0,2.0,20.8
4551,2024-05-19,Luton,2,4,0,15,6,1.0,3.2,0.2,8.2
4552,2024-05-19,Man City,3,1,3,28,12,3.4,0.2,3.0,13.8
4553,2024-05-19,Man United,2,0,3,11,4,1.6,2.0,1.4,18.0
4554,2024-05-19,Newcastle,4,2,3,12,7,2.4,1.6,1.4,17.8
4555,2024-05-19,Nott'm Forest,2,1,3,12,6,1.4,2.0,0.8,15.0
4556,2024-05-19,Sheffield United,0,3,0,6,1,1.0,3.4,0.0,14.6
4557,2024-05-19,Tottenham,3,0,3,18,9,1.2,2.4,0.6,15.0
4558,2024-05-19,West Ham,1,3,0,3,2,1.4,3.0,0.8,13.4
4559,2024-05-19,Wolves,0,2,0,4,3,0.8,2.4,0.6,9.6


In [69]:
# Features équipe domicile
features_dom = team_df.copy()
features_dom = features_dom.add_prefix("dom_")
features_dom.rename(columns={"dom_date_match":"date_match"}, inplace=True)

# Features équipe extérieur
features_ext = team_df.copy()
features_ext = features_ext.add_prefix("ext_")
features_ext.rename(columns={"ext_date_match":"date_match"}, inplace=True)


In [70]:
# Merge pour l’équipe domicile
df = df.merge(
    features_dom,
    left_on=["date_match", "equipe_dom"],
    right_on=["date_match", "dom_equipe"],
    how="left"
)

# Merge pour l’équipe extérieure
df = df.merge(
    features_ext,
    left_on=["date_match", "equipe_ext"],
    right_on=["date_match", "ext_equipe"],
    how="left"
)


In [71]:
df[[
    "date_match", "equipe_dom", "equipe_ext",
    "dom_points_moy_5", "ext_points_moy_5"
]].tail(10)


,date_match,equipe_dom,equipe_ext,dom_points_moy_5,ext_points_moy_5
2270,2024-05-19,Liverpool,Wolves,2.0,0.6
2271,2024-05-19,Crystal Palace,Aston Villa,2.8,1.6
2272,2024-05-19,Luton,Fulham,0.2,1.2
2273,2024-05-19,Chelsea,Bournemouth,2.8,1.2
2274,2024-05-19,Man City,West Ham,3.0,0.8
2275,2024-05-19,Brighton,Man United,1.0,1.4
2276,2024-05-19,Brentford,Newcastle,2.0,1.4
2277,2024-05-19,Arsenal,Everton,3.0,2.8
2278,2024-05-19,Burnley,Nott'm Forest,1.2,0.8
2279,2024-05-19,Sheffield United,Tottenham,0.0,0.6


In [72]:
df.columns

Index(['date_match', 'equipe_dom', 'equipe_ext', 'buts_dom', 'buts_ext',
       'resultat', 'tirs_dom', 'tirs_ext', 'tirs_cadres_dom',
       'tirs_cadres_ext', 'jaunes_dom', 'jaunes_ext', 'rouges_dom',
       'rouges_ext', 'cible_resultat', 'season', 'points_dom', 'points_ext',
       'dom_equipe', 'dom_buts_pour', 'dom_buts_contre', 'dom_points',
       'dom_tirs', 'dom_tirs_cadres', 'dom_buts_pour_moy_5',
       'dom_buts_contre_moy_5', 'dom_points_moy_5', 'dom_tirs_moy_5',
       'ext_equipe', 'ext_buts_pour', 'ext_buts_contre', 'ext_points',
       'ext_tirs', 'ext_tirs_cadres', 'ext_buts_pour_moy_5',
       'ext_buts_contre_moy_5', 'ext_points_moy_5', 'ext_tirs_moy_5'],
      dtype='object')

### Ecart domicile - exterieur

In [73]:
df["diff_points_moy_5"] = df["dom_points_moy_5"] - df["ext_points_moy_5"]
df["diff_buts_pour_moy_5"] = df["dom_buts_pour_moy_5"] - df["ext_buts_pour_moy_5"]
df["diff_buts_contre_moy_5"] = df["dom_buts_contre_moy_5"] - df["ext_buts_contre_moy_5"]
df["diff_tirs_moy_5"] = df["dom_tirs_moy_5"] - df["ext_tirs_moy_5"]


In [74]:
df[[
    "date_match", "equipe_dom", "equipe_ext",
    "diff_points_moy_5", "diff_buts_pour_moy_5",
    "diff_buts_contre_moy_5", "diff_tirs_moy_5"
]].head(10)


,date_match,equipe_dom,equipe_ext,diff_points_moy_5,diff_buts_pour_moy_5,diff_buts_contre_moy_5,diff_tirs_moy_5
0,2018-08-10,Man United,Leicester,NaN,NaN,NaN,NaN
1,2018-08-11,Bournemouth,Cardiff,NaN,NaN,NaN,NaN
2,2018-08-11,Fulham,Crystal Palace,NaN,NaN,NaN,NaN
3,2018-08-11,Huddersfield,Chelsea,NaN,NaN,NaN,NaN
4,2018-08-11,Newcastle,Tottenham,NaN,NaN,NaN,NaN
5,2018-08-11,Watford,Brighton,NaN,NaN,NaN,NaN
6,2018-08-11,Wolves,Everton,NaN,NaN,NaN,NaN
7,2018-08-12,Southampton,Burnley,NaN,NaN,NaN,NaN
8,2018-08-12,Liverpool,West Ham,NaN,NaN,NaN,NaN
9,2018-08-12,Arsenal,Man City,NaN,NaN,NaN,NaN


In [75]:
colonnes_a_supprimer = [
    "dom_points_moy_5","ext_points_moy_5",
    "dom_buts_pour_moy_5","ext_buts_pour_moy_5",
    "dom_buts_contre_moy_5","ext_buts_contre_moy_5",
    "dom_tirs_moy_5","ext_tirs_moy_5"
]

df = df.drop(columns=colonnes_a_supprimer)


In [76]:
df.to_csv("/content/premier_league_features.csv", index=False)


### Selection des colonnes utiles pour les modeles

In [81]:
features = [
    "diff_points_moy_5",
    "diff_buts_pour_moy_5",
    "diff_buts_contre_moy_5",
    "diff_tirs_moy_5",
]


In [78]:
target = "cible_resultat"


In [79]:
# Dates limites pour train / val / test
train_end = "2022-06-30"
val_end = "2023-06-30"

# Train : jusqu’au 30/06/2022
train_df = df[df["date_match"] <= train_end]

# Validation : 01/07/2022 → 30/06/2023
val_df = df[(df["date_match"] > train_end) & (df["date_match"] <= val_end)]

# Test : 01/07/2023 → fin (saison 23/24 ou 24/25)
test_df = df[df["date_match"] > val_end]


In [82]:
X_train = train_df[features]
y_train = train_df[target]

X_val = val_df[features]
y_val = val_df[target]

X_test = test_df[features]
y_test = test_df[target]


In [84]:
print("Entrainement:", X_train.shape, y_train.shape)
print("Validation:", X_val.shape, y_val.shape)
print("Test:", X_test.shape, y_test.shape)

X_train.head()
y_train.value_counts()


Entrainement: (1520, 4) (1520,)
Validation: (380, 4) (380,)
Test: (380, 4) (380,)


,count
cible_resultat,
2,660
0,526
1,334


In [87]:
X_train.to_csv("/content/ml/X_train.csv", index=False)
y_train.to_csv("/content/ml/y_train.csv", index=False)

X_val.to_csv("/content/ml/X_val.csv", index=False)
y_val.to_csv("/content/ml/y_val.csv", index=False)

X_test.to_csv("/content/ml/X_test.csv", index=False)
y_test.to_csv("/content/ml/y_test.csv", index=False)


Nous avons donc extrait nos fichiers pour l'entrainement, la validation et le test du modele.